In [1]:
import json
import time
import requests
import pandas as pd
from typing import List, Dict

In [2]:
df = pd.read_csv("top250.csv")
df.head()

,FilmId,Title,Rating,Year,Duration,Countries,Genres,ShortDescription,Director,Actors
0,435,Зеленая миля,9.1,1999,189,США,"драма, криминал",В тюрьме для смертников появляется заключенный...,Фрэнк Дарабонт,"Том Хэнкс, Дэвид Морс, Бонни Хант, Майкл Кларк..."
1,329,Список Шиндлера,8.8,1993,195,США,"драма, биография, военный, история","История немецкого промышленника, спасшего тыся...",Стивен Спилберг,"Лиам Нисон, Бен Кингсли, Рэйф Файнс, Кэролайн ..."
2,326,Побег из Шоушенка,9.1,1994,142,США,драма,Несправедливо осужденный банкир готовит побег ...,Фрэнк Дарабонт,"Тим Роббинс, Морган Фриман, Боб Гантон, Уильям..."
3,3498,Властелин колец: Возвращение короля,8.7,2003,201,"США, Новая Зеландия","драма, приключения, фэнтези","Арагорн штурмует Мордор, а Фродо устал боротьс...",Питер Джексон,"Элайджа Вуд, Вигго Мортенсен, Шон Эстин, Иэн М..."
4,448,Форрест Гамп,8.9,1994,142,США,"драма, мелодрама, комедия, военный, история",Полувековая история США глазами чудака из Алаб...,Роберт Земекис,"Том Хэнкс, Робин Райт, Салли Филд, Гэри Синиз,..."


In [3]:
# API-keys:
# 7cfc5665-f1c7-4569-9182-ab831fb3b25f
# 428ddb40-817c-4968-a9d2-cf8e1ae0a481
# f35b97c8-0485-4c4c-be05-ffd04d1ddfa2
# ee263835-92bb-44f4-a87f-64b1847ce734
# 78ef5609-9461-45e9-9393-456ad4bfc66f

In [14]:
root = "https://kinopoiskapiunofficial.tech"
headers = {
    "X-API-KEY": "428ddb40-817c-4968-a9d2-cf8e1ae0a481",
    "accept": "application/json"
}

In [15]:
def get_film(film_id):
    response = requests.get(f"{root}/api/v2.2/films/{film_id}", headers=headers)
    if response.status_code != 200:
        return Exception()
    return json.loads(response.content.decode())

In [16]:
def get_review_info(film_id, page=1, order="USER_POSITIVE_RATING_DESC"):
    params = {"page": page, "order": order}
    response = requests.get(f"{root}/api/v2.2/films/{film_id}/reviews", headers=headers, params=params)
    if response.status_code != 200:
        raise Exception()
    return json.loads(response.content.decode())

In [17]:
def get_film_review(film_id, pages, delay=1/10):
    res = []
    # reviews_info = get_review_info(film_id)
    # pages = reviews_info["totalPages"]
    for page in range(1, pages + 1):
        reviews_info = get_review_info(film_id, page=page)
        reviews: List[Dict] = reviews_info["items"]
        for r_info in reviews:
            review_data = list(r_info.values())
            review_data.insert(0, film_id)
            res.append(review_data)
        time.sleep(delay)
    return res

In [18]:
def get_films_reviews(film_ids, pages=1):
    data = []
    names = [
        "film_id", "review_id", "type", "date", "pos", "neg", 
        "author", "title", "description"
    ]
    try:
        for i, film_id in enumerate(film_ids, start=1):
            print(f"\r{i}/{len(film_ids)}", end="")
            reviews = get_film_review(film_id, pages)
            data.extend(reviews)
    except ConnectionError:
        print("\nError!")
    return pd.DataFrame(data=data, columns=names)

In [19]:
start_idx, end_idx = 0, 250
top_250_films_ids = df["FilmId"].to_list()
idxs = top_250_films_ids[start_idx:end_idx]
len(idxs), len(top_250_films_ids)

(249, 249)

In [20]:
top_250_films_reviews = get_films_reviews(idxs, pages=2)
top_250_films_reviews.to_csv("top_250_films_reviews.csv", index=False)
top_250_films_reviews

65/249

ConnectionError: HTTPSConnectionPool(host='kinopoiskapiunofficial.tech', port=443): Max retries exceeded with url: /api/v2.2/films/346/reviews?page=1&order=USER_POSITIVE_RATING_DESC (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001DAD91C41F0>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

In [27]:
data = get_film_review(435)
data

{'total': 469,
 'totalPages': 24,
 'totalPositiveReviews': 417,
 'totalNegativeReviews': 24,
 'totalNeutralReviews': 28,
 'items': [{'kinopoiskId': 65581,
   'type': 'POSITIVE',
   'date': '2007-06-26T19:47:00',
   'positiveRating': 485,
   'negativeRating': 63,
   'author': 'P.S.SeLeNa',
   'title': None,
   'description': '«Они помогли ему убить себя и так происходит каждый день во всем мире»…\r\n\r\nФильм, который я могу смело назвать шедевром. Фильм, который длится три часа и которые пролетают незаметно. Фильм, который смотрится каждый раз как в первый. Фильм, который нельзя забыть и которым нельзя не проникнуться. Фильм, котрый заставляет задуматься о жизни, которую мы тратим на пустяки, теша себя иллюзией бессмертия, о смерти, которая не щадит никого.\r\n\r\nФильм о зле, которое не всегда наказуемо и о добре, которое подчас бессильно что-либо изменить. И тебе остается лишь смотреть на то, как вершится суд, над тем кто не должен сидеть на скамье подсудимых. \r\n\r\nФильм, который 

In [30]:
len(data["items"])

20

In [52]:
items = data["items"]
items[0]["description"]

'«Они помогли ему убить себя и так происходит каждый день во всем мире»…\r\n\r\nФильм, который я могу смело назвать шедевром. Фильм, который длится три часа и которые пролетают незаметно. Фильм, который смотрится каждый раз как в первый. Фильм, который нельзя забыть и которым нельзя не проникнуться. Фильм, котрый заставляет задуматься о жизни, которую мы тратим на пустяки, теша себя иллюзией бессмертия, о смерти, которая не щадит никого.\r\n\r\nФильм о зле, которое не всегда наказуемо и о добре, которое подчас бессильно что-либо изменить. И тебе остается лишь смотреть на то, как вершится суд, над тем кто не должен сидеть на скамье подсудимых. \r\n\r\nФильм, который изменил мое мировоззрение и научил по-другому относиться к жизни.\r\n\r\n10 из 10'

In [25]:
get_film(435)

{'kinopoiskId': 435,
 'imdbId': 'tt0120689',
 'nameRu': 'Зеленая миля',
 'nameEn': None,
 'nameOriginal': 'The Green Mile',
 'posterUrl': 'https://kinopoiskapiunofficial.tech/images/posters/kp/435.jpg',
 'posterUrlPreview': 'https://kinopoiskapiunofficial.tech/images/posters/kp_small/435.jpg',
 'coverUrl': 'https://avatars.mds.yandex.net/get-ott/224348/2a00000169e39ef77f588ccdfe574dae8227/orig',
 'logoUrl': 'https://avatars.mds.yandex.net/get-ott/239697/2a0000016f12f1eb8870b609ee94313774b2/orig',
 'reviewsCount': 474,
 'ratingGoodReview': 93.2,
 'ratingGoodReviewVoteCount': 421,
 'ratingKinopoisk': 9.1,
 'ratingKinopoiskVoteCount': 813305,
 'ratingImdb': 8.6,
 'ratingImdbVoteCount': 1287059,
 'ratingFilmCritics': 6.8,
 'ratingFilmCriticsVoteCount': 136,
 'ratingAwait': None,
 'ratingAwaitCount': 0,
 'ratingRfCritics': None,
 'ratingRfCriticsVoteCount': 1,
 'webUrl': 'https://www.kinopoisk.ru/film/435/',
 'year': 1999,
 'filmLength': 189,
 'slogan': 'Пол Эджкомб не верил в чудеса. Пока 